# FABRIC API Example: Create a Simple Slice


## Configure the Environment

In [1]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

#os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRCI_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

#### Read the SSH Key File

In [2]:
slice_private_key_file=os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']
slice_public_key_file=os.environ['FABRCI_SLICE_PUBLIC_KEY_FILE']

slice_public_key = None
with open (slice_public_key_file, "r") as myfile:
    slice_public_key=myfile.read().strip()
    

## Setup the Experiment

#### Import FABRIC API

In [3]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

In [4]:
print(f"FABRIC_ORCHESTRATOR_HOST: {os.environ['FABRIC_ORCHESTRATOR_HOST']}")
print(f"FABRIC_CREDMGR_HOST:      {os.environ['FABRIC_CREDMGR_HOST']}")
print(f"FABRIC_TOKEN_LOCATION:    {os.environ['FABRIC_TOKEN_LOCATION']}")


slice_manager = SliceManager(oc_host=os.environ['FABRIC_ORCHESTRATOR_HOST'], 
                             cm_host=os.environ['FABRIC_CREDMGR_HOST'] ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

FABRIC_ORCHESTRATOR_HOST: orchestrator.fabric-testbed.net
FABRIC_CREDMGR_HOST:      cm.fabric-testbed.net
FABRIC_TOKEN_LOCATION:    /Users/pruth/work/fabric_token.json


## Create the Experiment Slice

#### Configure the Experiment Parameters



In [5]:
slice_name = 'MySlice'
site = 'MAX'
node_name = 'Node1'
username = 'centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 2
disk = 2

## Build the Slice

In [6]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType, Labels
# Create topology
experiment = ExperimentTopology()

# Add node
node = experiment.add_node(name=node_name, site=site)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
node.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Generate Slice Graph
slice_graph = experiment.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=slice_public_key)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Submitted slice creation request. Slice ID: 009b9a7e-3e2c-4682-a494-e42cfbf55806


In [7]:
# Generate Slice Graph
slice_graph = experiment.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                                         slice_graph=slice_graph, 
                                                         ssh_key=ssh_key_file_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

NameError: name 'ssh_key_file_pub' is not defined

## Query Slices

You can get a list of all your slices from the slice manager. If this is your first slice, it should return only one slice.

Note that the status returned by the call to slices indicates the success or failure of the call to the FABRIC control framework. The status is not the status of the slices.  The status of each slice is included in the list of slices.

In [8]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

MySlice:
   ID         : 009b9a7e-3e2c-4682-a494-e42cfbf55806
   State      : Configuring
   Lease End  : 2021-12-15 19:42:21



## Get the New Slice

You new slice is in the list of all your slices. You can loop through the list of slices to get the new slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

In [9]:
slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]

print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

Slice Name : MySlice
ID         : 009b9a7e-3e2c-4682-a494-e42cfbf55806
State      : Configuring
Lease End  : 2021-12-15 19:42:21


## Delete Slice

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))